In [ ]:
a = []
while(1):
    a.append('1')

In [ ]:
import torch
from transformers import IdeficsForVisionText2Text, AutoProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"

checkpoint = "HuggingFaceM4/idefics-9b-instruct"
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, torch_dtype=torch.bfloat16).to(device)
processor = AutoProcessor.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/99.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/7.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(model)

IdeficsForVisionText2Text(
  (model): IdeficsModel(
    (embed_tokens): IdeficsDecoupledEmbedding(
      num_embeddings=32000, num_additional_embeddings=3, embedding_dim=4096, partially_freeze=False
      (additional_embedding): Embedding(3, 4096)
    )
    (vision_model): IdeficsVisionTransformer(
      (embeddings): IdeficsVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(257, 1280)
      )
      (pre_layrnorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (encoder): IdeficsVisionEncoder(
        (layers): ModuleList(
          (0-31): 32 x IdeficsVisionEncoderLayer(
            (self_attn): IdeficsVisionAttention(
              (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
              (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
              (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
              (out_p

In [ ]:
def check_inference(model, processor, prompts, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=False)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

In [ ]:
url = "https://uploads8.wikiart.org/images/adriaen-brouwer/jan-davidszoon-de-heem.jpg"
prompts = [
    [
        "User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
        "https://uploads5.wikiart.org/images/adriaen-brouwer/farmers-fight-party.jpg",
        "<end_of_utterance>",
        "\nAssistant: Disgust.<end_of_utterance>",
        "\nUser:",
        url,
        "The man seems very content in this portrait. The man's face looks happy. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
        "\nAssistant:",
    ],
]

check_inference(model, processor, prompts, max_new_tokens=5)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: The man seems very content in this portrait. The man's face looks happy. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Contentment. 


### Data: Batch of 1k instances

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load Dataset
data = '/content/drive/My Drive/EC_Project/data/final_data.csv'
df = pd.read_csv(data)
df.head()

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance
0,aaron-siskind_acolman-1-1955,Abstract_Expressionism,fear,5,6,1,https://uploads7.wikiart.org/images/aaron-sisk...,Lost in a city and finding myself in a bad are...
1,aaron-siskind_chicago-1951,Abstract_Expressionism,fear,6,6,1,https://uploads5.wikiart.org/images/aaron-sisk...,looks like some sort of creature from a scary ...
2,aaron-siskind_chicago-6-1961,Abstract_Expressionism,fear,5,6,1,https://uploads7.wikiart.org/images/aaron-sisk...,The black sharp lines running throughout the c...
3,aaron-siskind_feet-102-1957,Abstract_Expressionism,disgust,5,5,1,https://uploads6.wikiart.org/images/aaron-sisk...,Foot looks weird and deformed. I hate feet
4,aaron-siskind_gloucester-16a-1944,Abstract_Expressionism,fear,5,6,1,https://uploads3.wikiart.org/images/aaron-sisk...,"it looks like an eye staring at me, or like a ..."


In [ ]:
# Emotion distribution and instance count
emotion_distribution = df['emotion'].value_counts(normalize=True)
total_instances = len(df)
print("Emotion Distribution:")
print(emotion_distribution)
print("Total Instances:", total_instances)

Emotion Distribution:
emotion
contentment    0.373313
awe            0.165090
amusement      0.131353
sadness        0.112814
fear           0.108210
excitement     0.062492
disgust        0.039507
anger          0.007220
Name: proportion, dtype: float64
Total Instances: 31716


In [ ]:
# Select 1k_batch
sample_size = 1000
sample_proportions = (emotion_distribution * sample_size).astype(int)

# select 1000 instances according to emotion label distribution
sample_df = df.groupby('emotion', group_keys=False).apply(lambda x: x.sample(sample_proportions[x.name]))

# randomly add up to 1000
while len(sample_df) < sample_size:
    diff = sample_size - len(sample_df)
    extra_samples = df.sample(diff)
    sample_df = pd.concat([sample_df, extra_samples])

while len(sample_df) > sample_size:
    sample_df = sample_df.sample(sample_size)

In [ ]:
sample_df.head()

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance
261,agostino-carracci_venus-and-mars-1600,Baroque,amusement,6,0,0,https://uploads7.wikiart.org/images/agostino-c...,All of the figures appear to have very importa...
6476,diego-velazquez_self-portrait-1,Baroque,amusement,5,0,0,https://uploads1.wikiart.org/images/diego-vela...,The mans collar looks like a college graduatio...
8422,felicien-rops_tienne-soubre,Symbolism,amusement,6,0,0,https://uploads6.wikiart.org/images/felicien-r...,The head is ridiculously large and the legs ar...
17538,koloman-moser_animal-motif-for-a-picture-book,Art_Nouveau_Modern,amusement,5,0,0,https://uploads6.wikiart.org/images/koloman-mo...,The little green shape reminds me of a bird an...
20142,martiros-saryan_woman-with-mask-s-i-dymshits-1913,Expressionism,amusement,5,0,0,https://uploads1.wikiart.org/images/martiros-s...,The facial expression indicates that woman is ...


In [ ]:
# Emotion distribution and instance count in sample_df
emotion_distribution = sample_df['emotion'].value_counts(normalize=True)
total_instances = len(sample_df)
print("Emotion Distribution (sample_df):")
print(emotion_distribution)
print("Total Instances(sample_df):", total_instances)

Emotion Distribution (sample_df):
emotion
contentment    0.374
awe            0.165
amusement      0.132
sadness        0.113
fear           0.108
excitement     0.062
disgust        0.039
anger          0.007
Name: proportion, dtype: float64
Total Instances(sample_df): 1000


In [ ]:
# save 1k_batch to google drive
sample_df.to_csv('/content/drive/My Drive/EC_Project/data/1k_batch.csv', index=False)

## Run Inference on 1k_batch

In [ ]:
# Load Sample Dataset
data = '/content/drive/My Drive/EC_Project/data/1k_batch.csv'
sample_df = pd.read_csv(data)
sample_df = sample_df.sample(frac=1).reset_index(drop=True) # shuffle the batch
sample_df.head(10)

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance
0,otto-eckmann_waldteich,Art_Nouveau_Modern,contentment,5,2,0,https://uploads1.wikiart.org/images/otto-eckma...,I like the colors and patterns in the picture....
1,konstantin-somov_models-2,Symbolism,contentment,5,2,0,https://uploads4.wikiart.org/images/konstantin...,the lines of the female form are fluid and not...
2,ernst-ludwig-kirchner_book-illustration-side-o...,Expressionism,awe,5,1,0,https://uploads8.wikiart.org/images/ernst-ludw...,It's amazing that these books have been preser...
3,salvador-dali_bather,Expressionism,contentment,11,2,0,https://uploads8.wikiart.org/images/salvador-d...,The woman is leaning back in her bed just like...
4,joseph-wright_landscape-with-figures-and-a-til...,Romanticism,awe,5,1,0,https://uploads3.wikiart.org/images/joseph-wri...,I feel awe looking at the hills and nature bey...
5,paul-gauguin_village-street-osny-1883,Impressionism,fear,5,6,1,https://uploads7.wikiart.org/images/paul-gaugu...,There's a dark sky behind this house and a dea...
6,hans-holbein-the-younger_portrait-of-thomas-cr...,Northern_Renaissance,awe,5,1,0,https://uploads4.wikiart.org/images/hans-holbe...,The dark design on the dark background actuall...
7,edvard-munch_from-thuringewald-1905,Expressionism,contentment,5,2,0,https://uploads5.wikiart.org/images/edvard-mun...,The contrast of colors reminds me of trips tak...
8,karl-bryullov_portrait-of-grand-duchess-elena-...,Romanticism,awe,5,1,0,https://uploads2.wikiart.org/images/karl-bryul...,the woman looks so angelic with her white dres...
9,francisco-goya_marquesa-mariana-de-pontejos,Romanticism,contentment,5,2,0,https://uploads4.wikiart.org/images/francisco-...,"Serene and peaceful, old painting country styl..."


In [ ]:
from tqdm import tqdm
import time

In [ ]:
# Iterate through each URL and run inference
results = []
start_time = time.time()

tqdm.pandas()

for index, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):
  utterance = row['utterance']
  url = row['Link']
  prompts = [
    [
        "User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
        "https://uploads5.wikiart.org/images/adriaen-brouwer/farmers-fight-party.jpg",
        "<end_of_utterance>",
        "\nAssistant: Disgust.<end_of_utterance>",
        "\nUser:",
        url,
        f"{utterance} Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
        "\nAssistant:",
    ],
]
  generated_text = check_inference(model, processor, prompts)
  print(generated_text)
  results.append(generated_text)

end_time = time.time()
elapsed_time = end_time - start_time

sample_df['generated_emotion_image+utterance'] = results

  0%|          | 1/1000 [00:04<1:09:45,  4.19s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: I like the colors and patterns in the picture. The pond surrounded by a forest represents a refuge in trying times. The blue of the pond and the stars remind me of a peaceful nature scene. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Contentment. 
User:

The image shows a woman in a red dress standing on a bridge. The woman is looking at the water and the moon. The moon is reflecting on the water. Choose one emotion from


  0%|          | 2/1000 [00:08<1:09:16,  4.16s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: the lines of the female form are fluid and not overly exaggerated the swift lines and soft lighting, as well as ease of the poses convey a sense of contentment Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Contentment. 
User:

Can you describe the overall theme of the drawing? 
Assistant: The overall theme of the drawing seems to be focused on the human form, particularly the female form, as it features multiple


  0%|          | 3/1000 [00:12<1:08:58,  4.15s/it]

...

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: So much more red thn any other colors, I'm not a fan of it at all. I don't like this painting because it is just an assortment of different colors layered on top of one another.  There are no shapes or figures to be seen. This feels amateur, like somebody was sampling their paint colors Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User:

What is the name of the painting? 
Assistant: The painting is called "Number 18". 
User:

Can you describe the painting's style


100%|█████████▉| 999/1000 [1:10:51<00:04,  4.71s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: The trees look really great in the sketch. The detail of the trees puts me in mind of being outside and just resting under one of these trees. nature always makes me feel content Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Contentment. 
User:

Can you describe the sketch of the trees? 
Assistant: The sketch features two trees, with a lot of detail in the bark and branches. The trees are depicted in a


100%|██████████| 1000/1000 [1:10:55<00:00,  4.26s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: The tiger's face is very funny and it made me laugh. The tiger looks like it just told you a bad joke. the face is comical and the colors are vibrate The tiger's round face and tuft of chest fur makes him look childlike and not at all fearsome. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Amusement. 
User:

The tiger's face is very funny and it made me laugh. The tiger looks like it just told you a bad joke. the face is comical and the colors are v


## Clean up the results

In [ ]:
import re

In [ ]:
def extract_second_assistant_emotion(text):
    # Find all occurrences of "Assistant: " in the text
    parts = re.split(r'Assistant: ', text)

    # If there are at least two occurrences
    if len(parts) > 2:
        # Extract the part after the second "Assistant: "
        second_assistant_part = parts[2]

        # Find the first emotion word from the specified list
        emotions = ['Amusement', 'Awe', 'Contentment', 'Excitement', 'Fear', 'Sadness', 'Anger', 'Disgust']
        for emotion in emotions:
            if emotion in second_assistant_part:
                return emotion

    # Return None if no emotion is found
    return None

In [ ]:
sample_df['second_assistant_emotion'] = sample_df['generated_emotion_image+utterance'].apply(extract_second_assistant_emotion)

In [ ]:
sample_df.head()

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance,generated_emotion_image+utterance,cleaned_emotion,second_assistant_emotion
0,otto-eckmann_waldteich,Art_Nouveau_Modern,contentment,5,2,0,https://uploads1.wikiart.org/images/otto-eckma...,I like the colors and patterns in the picture....,User: the figures are grotesque and animal-lik...,User: the figures are grotesque and animal-lik...,Contentment
1,konstantin-somov_models-2,Symbolism,contentment,5,2,0,https://uploads4.wikiart.org/images/konstantin...,the lines of the female form are fluid and not...,User: the figures are grotesque and animal-lik...,User: the figures are grotesque and animal-lik...,Contentment
2,ernst-ludwig-kirchner_book-illustration-side-o...,Expressionism,awe,5,1,0,https://uploads8.wikiart.org/images/ernst-ludw...,It's amazing that these books have been preser...,User: the figures are grotesque and animal-lik...,User: the figures are grotesque and animal-lik...,Anger
3,salvador-dali_bather,Expressionism,contentment,11,2,0,https://uploads8.wikiart.org/images/salvador-d...,The woman is leaning back in her bed just like...,User: the figures are grotesque and animal-lik...,User: the figures are grotesque and animal-lik...,None
4,joseph-wright_landscape-with-figures-and-a-til...,Romanticism,awe,5,1,0,https://uploads3.wikiart.org/images/joseph-wri...,I feel awe looking at the hills and nature bey...,User: the figures are grotesque and animal-lik...,User: the figures are grotesque and animal-lik...,Awe


In [ ]:
sample_df = sample_df.drop(['generated_emotion_image+utterance', 'cleaned_emotion'], axis=1)

In [ ]:
sample_df.head()

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance,second_assistant_emotion
0,otto-eckmann_waldteich,Art_Nouveau_Modern,contentment,5,2,0,https://uploads1.wikiart.org/images/otto-eckma...,I like the colors and patterns in the picture....,Contentment
1,konstantin-somov_models-2,Symbolism,contentment,5,2,0,https://uploads4.wikiart.org/images/konstantin...,the lines of the female form are fluid and not...,Contentment
2,ernst-ludwig-kirchner_book-illustration-side-o...,Expressionism,awe,5,1,0,https://uploads8.wikiart.org/images/ernst-ludw...,It's amazing that these books have been preser...,Anger
3,salvador-dali_bather,Expressionism,contentment,11,2,0,https://uploads8.wikiart.org/images/salvador-d...,The woman is leaning back in her bed just like...,None
4,joseph-wright_landscape-with-figures-and-a-til...,Romanticism,awe,5,1,0,https://uploads3.wikiart.org/images/joseph-wri...,I feel awe looking at the hills and nature bey...,Awe


In [ ]:
sample_df = sample_df.rename(columns={'second_assistant_emotion': 'generated_emotion'})

In [ ]:
sample_df.head()

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance,generated_emotion
0,otto-eckmann_waldteich,Art_Nouveau_Modern,contentment,5,2,0,https://uploads1.wikiart.org/images/otto-eckma...,I like the colors and patterns in the picture....,Contentment
1,konstantin-somov_models-2,Symbolism,contentment,5,2,0,https://uploads4.wikiart.org/images/konstantin...,the lines of the female form are fluid and not...,Contentment
2,ernst-ludwig-kirchner_book-illustration-side-o...,Expressionism,awe,5,1,0,https://uploads8.wikiart.org/images/ernst-ludw...,It's amazing that these books have been preser...,Anger
3,salvador-dali_bather,Expressionism,contentment,11,2,0,https://uploads8.wikiart.org/images/salvador-d...,The woman is leaning back in her bed just like...,None
4,joseph-wright_landscape-with-figures-and-a-til...,Romanticism,awe,5,1,0,https://uploads3.wikiart.org/images/joseph-wri...,I feel awe looking at the hills and nature bey...,Awe


In [ ]:
sample_df.to_csv('/content/drive/My Drive/EC_Project/data/1k_generation0.csv', index=False)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

In [ ]:
# Drop rows where 'generated_emotion' or 'emotion' is NaN
sample_df = sample_df.dropna(subset=['generated_emotion', 'emotion'])

In [ ]:
sample_df['generated_emotion'] = sample_df['generated_emotion'].str.lower()

In [ ]:
# Calculate accuracy, F1 score, precision, recall
accuracy = accuracy_score(sample_df['emotion'], sample_df['generated_emotion'])
f1 = f1_score(sample_df['emotion'], sample_df['generated_emotion'], average='weighted')
precision = precision_score(sample_df['emotion'], sample_df['generated_emotion'], average='weighted')
recall = recall_score(sample_df['emotion'], sample_df['generated_emotion'], average='weighted')
report = classification_report(sample_df['emotion'], sample_df['generated_emotion'])

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print("Classification Report:")
print(report)

Accuracy: 0.6947040498442367
F1 Score: 0.7134879957775526
Precision: 0.7855614852929684
Recall: 0.6947040498442367
Classification Report:
              precision    recall  f1-score   support

   amusement       0.42      0.87      0.57       127
       anger       0.04      0.14      0.06         7
         awe       0.93      0.51      0.66       158
 contentment       0.89      0.75      0.82       353
     disgust       0.42      0.69      0.52        39
  excitement       0.56      0.37      0.44        60
        fear       0.78      0.69      0.73       106
     sadness       0.95      0.78      0.85       113

    accuracy                           0.69       963
   macro avg       0.63      0.60      0.58       963
weighted avg       0.79      0.69      0.71       963



## Input only text data, using blank image as place holder

In [ ]:
# URLs
url = 'https://via.placeholder.com/200' # 200*200 pixel
#url = 'https://via.placeholder.com/100' # 100*100 pixel

## Input only image data, using blank text as place holder